In [43]:

# 设置变量表单项的变量会被预置到一个名为 bundle['inputData'] 的对象中，
# 例如 bundle['inputData']['name'] 就可以访问到变量 name
# output 是此节点能够识别的输出，如果不对它赋值，程序可能异常或非预期
import requests
import json
def get_id_database(title):
    payload_query = {"filter": {"property": "任务名", "title":{"equals":title}}}
    base_url = "https://api.notion.com/v1/databases/"
    Database_ID = '06d9339bbd8f48e59dc7d77f5a9ae1e8'
    headers = {
        "Authorization": "secret_O0Hh83N3sInLqbpEn0TocYNNQpjkU8mrrLBre3ab3yM",
        "accept": "application/json",
        "Notion-Version": "2022-06-28",
    }
    response = requests.post(base_url + Database_ID + "/query", headers=headers, json=payload_query)
    jst = json.loads(response.text)
    print(jst)
    id = jst['results'][0]['id']
    working_days = jst['results'][0]['properties']['实际工时(天)']['number']
    if not jst['results'][0]['properties']['进展记录']['rich_text']:
        progress = 0
    else:
        progress = jst['results'][0]['properties']['进展记录']['rich_text'][0]['plain_text']
    if working_days is None:
        working_day = 0
    return {'id':id,'任务名':title,'working_days':working_days,'progress':progress}

output=get_id_database('财务Payment-自动化流程')
print(output)

{'object': 'list', 'results': [{'object': 'page', 'id': '6685e37e-fa62-427f-aa2d-d28bd8e22a41', 'created_time': '2022-10-19T10:40:00.000Z', 'last_edited_time': '2022-10-26T07:52:00.000Z', 'created_by': {'object': 'user', 'id': 'feed7a92-be24-4351-b50f-2fae861d445d'}, 'last_edited_by': {'object': 'user', 'id': 'e5eb5fda-2756-4e1f-a7b3-1f4ac4161b00'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '06d9339b-bd8f-48e5-9dc7-d77f5a9ae1e8'}, 'archived': False, 'properties': {'预计完成时间': {'id': '%40%5C%40B', 'type': 'date', 'date': {'start': '2022-10-26', 'end': None, 'time_zone': None}}, '创建时间': {'id': 'FMx%3E', 'type': 'created_time', 'created_time': '2022-10-19T10:40:00.000Z'}, '进展记录': {'id': 'Hqm%3B', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': '221025测试3', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '221025测试3', 'href': 

In [42]:
bundle = {'inputData':{'working_days':0,'status':'未启动','id':'6685e37e-fa62-427f-aa2d-d28bd8e22a41','progress':'0','task_content':'测试3','date':'221025'}}
import requests
working_days = bundle['inputData']['working_days']
Page_ID = bundle['inputData']['id']
progress = bundle['inputData']['progress']
task_content = bundle['inputData']['task_content']
date = bundle['inputData']['date']
base_url = "https://api.notion.com/v1/pages/"
if progress == '0' and task_content is None:
    content = ''
elif progress == '0' and task_content is not None:
    content = date+task_content
elif progress != '0' and task_content is not None:
    content = progress+'\n'+date+task_content
else:
    content = progress

headers = {
    "Authorization": "secret_O0Hh83N3sInLqbpEn0TocYNNQpjkU8mrrLBre3ab3yM",
    "accept": "application/json",
    "content-type": "application/json",
    "Notion-Version": "2022-06-28",
}
payload = {
    "properties": {
        '实际工时(天)': {
            "number": float(working_days)+0.5
        },'是否完成':{
            "status": {'id': '0ed43941-1073-4ba7-882a-61bc13fe791b', 'name': '进行中', 'color': 'blue'}
        },'进展记录':{'id': 'Hqm%3B', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': content, 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': content, 'href': None}]}
    }
}
response = requests.patch(url=base_url + Page_ID, headers=headers,json=payload)
output={'working_days':working_days}

In [8]:
import requests
import json
payload = {"filter": {"or":[{"property": "是否完成", "status":{"equals":"进行中"}},{"property": "是否完成", "status":{"equals":"未启动"}}]}}
base_url = "https://api.notion.com/v1/databases/"
Database_ID = '06d9339bbd8f48e59dc7d77f5a9ae1e8'
headers = {
    "Authorization": "secret_O0Hh83N3sInLqbpEn0TocYNNQpjkU8mrrLBre3ab3yM",
    "accept": "application/json",
    "Notion-Version": "2022-06-28",
}
response = requests.post(base_url + Database_ID + "/query", headers=headers, json=payload)
jst = json.loads(response.text)
title_list_all = {"options":[]}
num = 1
for i in jst['results']:
    title_list={}
    if i['properties']['优先级']['select']['name']=='高优先级':
        title_list['task_优先级']=5
    elif i['properties']['优先级']['select']['name']=='中优先级':
        title_list['task_优先级']=3
    elif i['properties']['优先级']['select']['name']=='低优先级':
        title_list['task_优先级']=1
    else:
        title_list['task_优先级']=0
    title_list['task_任务名']=i['properties']['任务名']['title'][0]['text']['content']
    title_list_all["options"].append(title_list)
    num+=1
title_list_all["options"].sort(key=lambda s: s["task_优先级"],reverse=True)
print(title_list_all)

{'options': [{'task_优先级': 5, 'task_任务名': 'WF爬虫2.0_新增库存信息'}, {'task_优先级': 3, 'task_任务名': '财务Payment-自动化流程'}, {'task_优先级': 1, 'task_任务名': '仓库的数据输出'}, {'task_优先级': 1, 'task_任务名': '爬取零售价的价位区间'}, {'task_优先级': 1, 'task_任务名': '仓储费用计算'}]}


In [7]:
title_list = [{'task_优先级': 1, 'task_任务名': '仓库的数据输出'}, {'task_优先级': 1, 'task_任务名': '爬取零售价的价位区间'}, {'task_优先级': 1, 'task_任务名': '仓储费用计算'}, {'task_优先级': 5, 'task_任务名': 'WF爬虫2.0_新增库存信息'}, {'task_优先级': 3, 'task_任务名': '财务Payment-自动化流程'}]
title_list.sort(key=lambda s: s["task_优先级"],reverse=True)

print(title_list)

[{'task_优先级': 5, 'task_任务名': 'WF爬虫2.0_新增库存信息'}, {'task_优先级': 3, 'task_任务名': '财务Payment-自动化流程'}, {'task_优先级': 1, 'task_任务名': '仓库的数据输出'}, {'task_优先级': 1, 'task_任务名': '爬取零售价的价位区间'}, {'task_优先级': 1, 'task_任务名': '仓储费用计算'}]


In [19]:
import requests
task_name = '自动到仓邮件截图'
properties = '是否完成'
'https://www.notion.so/4c3f34849ad74390b70de60354c18f37'
base_url = "https://api.notion.com/v1/pages/"
Page_ID = '4c3f34849ad74390b70de60354c18f37'
headers = {
    "Authorization": "secret_O0Hh83N3sInLqbpEn0TocYNNQpjkU8mrrLBre3ab3yM",
    "accept": "application/json",
    "content-type": "application/json",
    "Notion-Version": "2022-06-28",
}
payload = {
    "properties": {
        properties: {
            "status": {'id': '0ed43941-1073-4ba7-882a-61bc13fe791b', 'name': '进行中', 'color': 'blue'}
        }
    }
}
response = requests.patch(url=base_url + Page_ID, headers=headers,json=payload)

In [ ]:
import logging
logger_name = 'logger'
handler_name = 'handler'
logger = logging.getLogger(logger_name)
logger.setLevel(logging.ERROR)
logger.addHandler(handler_name) # 为Logger实例增加一个处理器

ch.setFormatter(formatter_name) # 设置一个格式化器formatter
ch.addFilter(filter_name) # 增加一个过滤器，可以增加多个
ch.removeFilter(filter_name) # 删除一个过滤器

In [13]:
import pandas as pd
import os

g = os.walk(r"E:\OneDrive\广新\CG账单")
i = 1
full_path_list = []
for path,dir_list,file_list in g:
    for file_name in file_list:
        full_path = os.path.join(path, file_name)
        if 'csv' in str(file_name):
            print(full_path)
            full_path_list.append(full_path)
        i+=1
print(full_path_list)

E:\OneDrive\广新\CG账单\Invoice_113007_20220503104237.csv
E:\OneDrive\广新\CG账单\Invoice_121249_20220503104828.csv
E:\OneDrive\广新\CG账单\Invoice_121431_20220503104823.csv
E:\OneDrive\广新\CG账单\Invoice_121614_20220503104818.csv
E:\OneDrive\广新\CG账单\Invoice_121813_20220503104813.csv
E:\OneDrive\广新\CG账单\Invoice_122005_20220503104807.csv
E:\OneDrive\广新\CG账单\Invoice_122213_20220503104800.csv
E:\OneDrive\广新\CG账单\Invoice_122428_20220503104754.csv
E:\OneDrive\广新\CG账单\Invoice_122638_20220503104748.csv
E:\OneDrive\广新\CG账单\Invoice_122895_20220503104737.csv
E:\OneDrive\广新\CG账单\Invoice_123158_20220503104722.csv
E:\OneDrive\广新\CG账单\Invoice_123442_20220503104712.csv
E:\OneDrive\广新\CG账单\Invoice_123752_20220503104701.csv
E:\OneDrive\广新\CG账单\Invoice_124010_20220503104655.csv
E:\OneDrive\广新\CG账单\Invoice_124100_20220503104650.csv
E:\OneDrive\广新\CG账单\Invoice_124430_20220503104642.csv
E:\OneDrive\广新\CG账单\Invoice_124783_20220503104258.csv
E:\OneDrive\广新\CG账单\Invoice_124783_20220503104635.csv
E:\OneDrive\广新\CG账单\Invoice_

In [9]:
# coding:utf-8
'''
@File    : robot.py
@Author  : @VTester
@Desc    : 企业微信机器人
'''

import requests
import json
import base64
import hashlib

def robot(key, data):
    """_summary_

    Args:
        key (_type_): _description_
        data (_type_): _description_
    """
    webhook = f"https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key={key}"
    # 请求头
    headers = {'content-type': 'application/json'}
    r = requests.post(webhook, headers=headers, json=data)
    r.encoding = 'utf-8'
    print(f'执行内容:{data}, 参数:{r.text}')
    print(f'webhook 发送结果:{r.text}')
    return r.text

def bot_push(key, data):
    """发送请求结果

    Args:
        key (_type_): _description_
        data (_type_): _description_

    Returns:
        _type_: _description_
    """
    try:
        res = robot(key, data)
        print(res)  # 打印请求结果
        print(f'webhook 发出完毕: {res}')
        return res
    except Exception as e:
        print(e)

def bot_push_text(key, msg, mobile_list):
    """发送文本

    Args:
        key (_type_): _description_
        msg (_type_): _description_
    """
    webhook_data = {
        "msgtype": "text",
        "text":{
            "content": msg,
            "mentioned_mobile_list":mobile_list
        }
    }

    # 机器人发送
    bot_push(key, webhook_data)
    return None

def bot_push_image(key):
    """发送图片

    Args:
        key (_type_): _description_
        msg (_type_): _description_
    """

    # 图片base64码
    with open(r"E:\\OneDrive\图片\WeiXin\mmexport1643682690754.jpg","rb") as f:
        base64_data = base64.b64encode(f.read())
    # base64.b64decode(base64data)
    # print(base64_data)

    # 图片的md5值
    file = open(r"E:\\OneDrive\图片\WeiXin\mmexport1643682690754.jpg", "rb")
    md = hashlib.md5()
    md.update(file.read())
    res1 = md.hexdigest()
    # print(res1)

    webhook_data = {
    "msgtype": "image",
    "image": {
        "base64": base64_data,
        "md5": res1
    }
}

    # 机器人发送
    # bot_push(key, webhook_data)
    res = robot(key, webhook_data)
    return None

daily_list = []
msg1 = "日期：{0[0]}\n晓望集群单量：{0[1]}\n晓望集群营业额：{0[2]}\n晓望集群利润：{0[3]}\n晓望集群库存量：{0[4]}\n晓望集群库存周转天数：{0[5]}".format(daily_list)
msg2 = '请发送日报截图和简报数据到微信群'
mobile_list1 = []
mobile_list2 = ['18062351119']
bot_push_text("23214d5d-a5af-4be3-b8f2-d8ca154e0b57", msg1, mobile_list1)
bot_push_text("23214d5d-a5af-4be3-b8f2-d8ca154e0b57", msg2, mobile_list2)
# bot_push_image("23214d5d-a5af-4be3-b8f2-d8ca154e0b57")

IndexError: Replacement index 0 out of range for positional args tuple

In [14]:
from PIL import Image
import os
import pytesseract
import cv2 as cv
import fitz

ImportError: cannot import name 'ParamSpec' from 'typing_extensions' (C:\Users\Admin\anaconda3\lib\site-packages\typing_extensions.py)

In [23]:
def pdf_image(pdfPath,imgPath,zoom_x,zoom_y,rotation_angle):
    # 打开PDF文件
    pdf = fitz.open(pdfPath)
    # 逐页读取PDF
    for pg in range(0, pdf.pageCount):
        page = pdf[pg]
        # 设置缩放和旋转系数
        trans = fitz.Matrix(zoom_x, zoom_y).preRotate(rotation_angle)
        pm = page.getPixmap(matrix=trans, alpha="False")
        # 开始写图像
        pm.writePNG(imgPath+str(pg)+".png")
        #pm.writePNG(imgPath)
    pdf.close()
pdf_path ='C:/Users/Admin/Documents/Tencent Files/544409644/FileRecv/CREDIT APPLICATION FORM SAMPLE.pdf'
img_path ='C:/Users/Admin/Documents/Tencent Files/544409644/FileRecv/CREDIT APPLICATION FORM SAMPLE'
# pdf_image(pdf_path,img_path,5,5,0)
img_path = img_path+"0.png"
# 依赖opencv
# img=cv.imread(img_path)
# text=pytesseract.image_to_string(Image.fromarray(img),lang='eng')
# 不依赖opencv写法
text=pytesseract.image_to_string(Image.open(img_path))
print(text)

50 East Wilmot St. e
Rich d Hill, Ontario, Canada, L4B 3Z3 k
hand Fil, Ontario, Canad C.J. Mar eting Ltd.

Fax: (905) 889-8982 os ‘ :
(908) E-Mail: info@cjmarketing.com

CREDIT APPLICATION AND AGREEMENT ( PAGE 1 OF 2 )

| (WE) ACCEPTS AS NOTICE IN WRITING OF AND CONSENTS TO THE OBTAINING FROM ANY CREDIT REPORTING
AGENCY OR ANY PERSON SUCH INFORMATION AS C. J. MARKETING LTD. MAY REQUIRE AT ANY TIME IN
CONNECTION WITH THE CREDIT HEREBY APPLIED FOR AND CONSENTS TO THE DISCLOSURE OF ANY
INFORMATION CONCERNING THE UNDERSIGNED TO ANY CREDIT AGENCY, OR TO ANY PERSON WITH WHOM THE
UNDERSIGNED HAS OR MAY HAVE FINANCIAL RELATIONS.

THE UNDERSIGNED AGREES TO THE FOLLOWING:
1) PAY ACCORDING TO THE TERMS OF SALE IN EFFECT AT TIME OF PURCHASE.
2) PAY SERVICE CHARGES OF 1.5% PER MONTH (19.56 % PER ANNUM) ON ANY AMOUNTS OVERDUE.

3.) ACCEPT THAT PAYMENTS ARE TO BE ATTRIBUTED FIRSTLY TO THE SERVICE COSTS AND THEN TO THE PAYMENT
OF THE PURCHASES IN THEIR CHRONOLOGICAL ORDER.

4) ACCEPT THAT IF AT ANY TI

In [24]:
with open(r'C:\Users\Admin\Documents\Tencent Files\544409644\FileRecv\0.txt','w') as f:
    f.write(text)
    f.close()

In [16]:
import pandas as pd
import xlwings as xw
app = xw.apps.add()
book = app.books.open(r'E:\OneDrive\广新\供应链-Crystal\晓望梅观 2021.1月港前费RMB账单.xlsx',read_only="True")
sheet = book.sheets[0]

In [17]:
import os
def bianli(rootDir):
    os_list = []
    for root,dirs,files in os.walk(rootDir):
        for file in files:
            os_path = os.path.join(root,file)
            os_list.append(os_path)
        for dir in dirs:
            bianli(dir)
    return os_list

In [1]:
dict1 = {}
if dict1:
    print('true')
else:
    print('False')

False


In [2]:
import sys
import time
for i in range(1, 101):
  print("\r", end="")
  print("进度: {}%: ".format(i), "▓" * (i // 2), end="")
  sys.stdout.flush()
  time.sleep(0.05)

进度: 100%:  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

In [4]:
import tqdm
from time import sleep
pbar = tqdm.tqdm(total=100)
for i in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()

100%|██████████| 100/100 [00:01<00:00, 90.05it/s]


In [ ]:
pbar = tqdm(total = runs+1)
while currentData[0] <= runs:

    ### ROLLING THE DICES PROCESS ###
    dices = twinDiceRoll()
    currentData[1] += dices[2]  # Updating the current tile

    ### SURPASSING THE NUMBER OF TILES ONBOARD ###
    if currentData[1] > 37:   # If more than a table turn is achieved,
        currentData[0] += 1   # One more turn is registered
        currentData[1] -= 38  # Update the tile to one coresponding to a board tile.
        pbar.update(1)
    else:
        pass
...
pbar.close()

In [8]:
import Wayfair_Scrapy

In [14]:
from multiprocessing import Pool
import os

def f(x):
    print('Child process id:', os.getpid())
    return x*2

if __name__ == '__main__':
    print('Parent process id:', os.getpid())
    pool = Pool(4)
    data_list = pool.map(f, [1,2,3])

Parent process id: 27304


Process SpawnPoolWorker-41:
Process SpawnPoolWorker-39:
Process SpawnPoolWorker-42:
Traceback (most recent call last):
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'f' on <module '__main__' (built-in)>
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessin

KeyboardInterrupt: 

ject/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/Users/huzhang/.conda/envs/pythonProject/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [ ]:
result = list(tqdm(pool.imap(partial(func,y = math.pi), np.linspace(0,2*math.pi,1000)), total=1000))
